In [81]:
import sklearn
import numpy as np
import pandas as pd
import os
from scipy import signal

from sklearn.model_selection import train_test_split

#### Assuming that the data is already normalised

In [2]:
# Creating the different sets 
training_set = []
testing_set = []
validation_set = []

# Choosing to create a dataframe that stores the numpy array and the action name
data = pd.DataFrame(columns= ['action_id', 'action', 'label'])
#data_norm = pd.DataFrame(columns= ['action_id_norm', 'action'])

In [3]:
path_action_down = 'actions/action_down'
path_action_up = 'actions/action_up'
path_action_switch = 'actions/action_switch'
path_arr = [path_action_down, path_action_up, path_action_switch]


action_id = []
action = []
label_action = []
action_id_norm = []

# Normalise the data
def min_max_normalise(arr):
    min = np.min(arr)
    #print(min)
    max = np.max(arr)
    #print(max)
    normalise = (arr - min)/(max-max)
    print(arr)
    print(normalise)
    return normalise

# for action in the path, get name and array and add to the dataframe
# Work from the dataset
for path_type in path_arr:
    for file in os.listdir(path_type):
        f = os.path.join(path_type, file)
        heatmap = np.load(f)

        '''
        #normalise
        normalise = min_max_normalise(heatmap)
        if not os.path.exists('normalised_data'):    
            os.makedirs('normalised_data')
        norm_path = os.path.join('normalised_data', file)
        save_norm = np.save(norm_path, normalise)
        '''

        action_id.append(file)
        action.append(path_type)

        if path_type is path_arr[0]:
            label = 'D'
        if path_type is path_arr[1]:
            label = 'U'
        if path_type is path_arr[2]:
            label = 'S'

        label_action.append(label)
        #action_id_norm.append(save_norm)

data['action_id'] = action_id
data['action'] = action
data['label'] = label_action

#data_norm['action_id_norm'] = action_id_norm
#data_norm['action'] = action

In [4]:
data.head()

,action_id,action,label
0,action_down_1610_0.npy,actions/action_down,D
1,action_down_1655_0.npy,actions/action_down,D
2,action_down_547_0.npy,actions/action_down,D
3,action_down_502_0.npy,actions/action_down,D
4,action_down_990_0.npy,actions/action_down,D


In [55]:
data.shape

(7485, 3)

In [ ]:
class_names = ['down', 'up', 'switch']

#### Train and Test Split

In [5]:
# array x is array of npy arra<ys and Y is array of labels
X = []
Y = label_action

for path_type in path_arr:
    for file in os.listdir(path_type):
        f = os.path.join(path_type, file)
        heatmap = np.load(f)
        X.append(heatmap)

print(len(X))
print(len(Y))

7485
7485


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size= 0.2, random_state=42)

In [7]:
print(len(X_train))
print(len(X_test))

5988
1497


In [97]:
all_train = np.empty((5988, 17, 64, 48))
for i in range(0, len(X_train)):
    all_train[i] = X_train[i]
print(all_train.shape)

(5988, 17, 64, 48)


### Architecture of CNN
Convolution + ReLU --> Pooling --> Convolution + ReLU --> Pooling -->...--> Flatten --> Fully connected --> softmax 

### Convolutional Operation (filter) and Activation function

In [106]:
stride = 1
# Choose a filter
conv1 = np.random.randn(1,17,4,4) * np.sqrt(1. / 5)
filter_w = conv1.shape[3]
filter_h = conv1.shape[2]
# Create resulting matrix R
image_w = 48
image_h = 64
result_w = (int) ((image_w-filter_w)/stride)+1
result_h = (int) ((image_h-filter_h)/stride)+1

In [ ]:
# RELU - learn complex patterns
def ReLU(x):
    return (x>0)*x

In [114]:
correlations_per_filter = np.empty((5988, 17, result_h, result_w))
for image in X_train: # Change this to all data
    for f in range(0, len(image[0])):
        conv_filter = conv1[0][f]
        image_filter = image[0][f]
        correlation_valid = signal.correlate2d(image_filter, conv_filter, mode='valid')
        activated = ReLU(correlation_valid)
        correlations_per_filter[f] = activated

print(correlations_per_filter.shape)

(5988, 17, 61, 45)


In [116]:
correlations_per_filter[0][1]

array([[ 3.20323510e-04,  1.05567382e-04,  6.57867487e-04, ...,
         4.82129533e-03, -0.00000000e+00,  3.52744625e-04],
       [ 1.33448774e-04,  3.87780086e-04,  2.76501404e-04, ...,
         2.68109299e-03,  1.12663092e-02, -0.00000000e+00],
       [-0.00000000e+00,  2.31607803e-04,  5.94882991e-04, ...,
         2.95992649e-03,  1.69868343e-03,  7.68081595e-03],
       ...,
       [ 7.62618692e-04,  1.21867822e-04,  1.22213459e-04, ...,
         1.22408927e-04,  1.43028246e-04,  1.10342333e-04],
       [ 6.28075042e-04,  1.23760500e-04,  1.22018199e-04, ...,
         1.19846287e-04,  9.81622508e-05,  1.25086298e-04],
       [ 7.63139474e-04,  6.77027749e-04,  6.07475328e-05, ...,
         1.03399769e-04, -0.00000000e+00,  1.84674453e-04]])

In [109]:
def ReLU_two(x):
    return max(0,x)

### Pooling

In [138]:
import tensorflow as tf

def perform_max_pool(single_correlation, w, h, pool_size, stride):
    reshaped = tf.reshape(single_correlation, [1,h,w,17])
    max_pool = tf.keras.layers.MaxPooling2D(pool_size, strides=stride, padding='valid')
    return max_pool(reshaped)

In [139]:
# Max pooling - Downsize feature map - extract lowlevel features 
# Experiments: Keras has maxpooling2d, averagepooling2d
# Hyper parameter tuning - stride and pooling size
max_pool_stride = 2
pool_size = 2

input_width = correlations_per_filter.shape[3]
input_height = correlations_per_filter.shape[2]
    
output_width = int((input_width-pool_size)/max_pool_stride)+1
output_height = int((input_height-pool_size)/max_pool_stride)+1
    
pools = np.zeros((correlations_per_filter.shape[0],output_height,output_width, correlations_per_filter.shape[1]))

for i in range(0, correlations_per_filter.shape[0]):
    pools[i] = perform_max_pool(correlations_per_filter[i], input_width, input_height, (2,2), (2,2))

In [141]:
pools[0].shape

(30, 22, 17)

In [ ]:
correlations_per_filter = np.empty((5988, 17, result_h, result_w))

def convolution_step(conv1, images):
    for image in images: # Change this to all data
        for f in range(0, len(image[0])):
            conv_filter = conv1[0][f]
            image_filter = image[0][f]
            correlation_valid = signal.correlate2d(image_filter, conv_filter, mode='valid')
            activated = ReLU(correlation_valid)
            correlations_per_filter[f] = activated

print(correlations_per_filter.shape)

### Back Propagation - Fully connected layers (Maybe watch video part as well)

### Multi-Class classification - Multiple neurons with a softmax activation

### Loss Function

### Training

In [ ]:
# Epochs - cycles of training the NN with all the training data
# Pooling - reduce size of image
# Classification - fully connected layer - dense layer with output for each class

### Testing